<a href="https://colab.research.google.com/github/andersoncruzz/SSFER/blob/master/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`Google Drive Module`**

In [0]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# class Drive:
#     def __init__(self):
#         auth.authenticate_user()
#         self.gauth = GoogleAuth()
#         self.gauth.credentials = GoogleCredentials.get_application_default()
#         self.drive = GoogleDrive(self.gauth)
        
#     def create_dir(self, top, name):
#         if top == "root":
#             new_folder = self.drive.CreateFile({'title': name, \
#                                       "mimeType": "application/vnd.google-apps.folder"})
#             new_folder.Upload() 
#             return new_folder
        
#         file_list = self.drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
#         for file in file_list:
#             if file["title"] == top:
#                 print("found top")
#                 break
#         print('title: %s, id: %s' % (file['title'], file['id']))
#         new_folder = self.drive.CreateFile({'title': name, \
#                                     "parents":  [{"id": file["id"]}], \
#                                     "mimeType": "application/vnd.google-apps.folder"})
#         new_folder.Upload()
#         return new_folder

In [0]:
!cat /proc/meminfo

MemTotal:       13335212 kB
MemFree:        11568560 kB
MemAvailable:   12670764 kB
Buffers:           41128 kB
Cached:          1237336 kB
SwapCached:            0 kB
Active:           442304 kB
Inactive:        1120728 kB
Active(anon):     264844 kB
Inactive(anon):      316 kB
Active(file):     177460 kB
Inactive(file):  1120412 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               144 kB
Writeback:             0 kB
AnonPages:        284564 kB
Mapped:           152560 kB
Shmem:               820 kB
Slab:             117504 kB
SReclaimable:      91984 kB
SUnreclaim:        25520 kB
KernelStack:        3664 kB
PageTables:         4628 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667604 kB
Committed_AS:    1794700 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

# **`Common class`**


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

class Common:
    def save_plot_losses(self, history, path_dir):
        plt.figure(figsize=[8,6])
        plt.plot(history.history['loss'],'r',linewidth=3.0)
        plt.plot(history.history['val_loss'],'b',linewidth=3.0)
        plt.legend(['Custo no Treino', 'Custo no Teste'],fontsize=18)
        plt.xlabel('Épocas ',fontsize=16)
        plt.ylabel('Custo',fontsize=16)
        plt.title('Curvas da Função de Custo',fontsize=16)
        plt.savefig(os.path.join(path_dir, 'loss.png'))

    def save_plot_acc(self, history, path_dir):
        plt.figure(figsize=[8,6])
        plt.plot(history.history['acc'],'r',linewidth=3.0)
        plt.plot(history.history['val_acc'],'b',linewidth=3.0)
        plt.legend(['Acurácia no Treino', 'Acurácia no Teste'],fontsize=18)
        plt.xlabel('Épocas ',fontsize=16)
        plt.ylabel('Acurácia',fontsize=16)
        plt.title('Curvas da Acurácia',fontsize=16)
        plt.savefig(os.path.join(path_dir, 'acc.png'))

    def write_file_info(self, path_dir, content):
        file_info_path = os.path.join(path_dir, "info.txt")
        file_info_experiment = open(file_info_path, "a+")

        for i in content:
            line = str(i) + "\n"
            file_info_experiment.write(line)

        file_info_experiment.close()

    def train(self, model, x_train, y_train, x_test, y_test, batch_size, epochs, path_dir):
        self.write_file_info(path_dir, ["batch_size: "+str(batch_size)])

        file_weights = os.path.join(path_dir, "weights.{epoch:02d}-{val_loss:.2f}.hdf5")
        checkpoint = ModelCheckpoint(file_weights, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
#         checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=1, mode='auto')

        callbacks_list = [checkpoint, earlystopping]

        history = model.fit(x_train, y_train,
                    batch_size = 20,
                    epochs = epochs,
                    verbose = 1,
#                     steps_per_epoch = 100,
                    shuffle = True,
                    validation_data=(x_test, y_test),
                    callbacks = callbacks_list)

        return model, history

    def evaluate(self, model, x_test, y_test):
        score = model.evaluate(x_test, y_test, verbose=0)
        return score

    def compile_net(self, model, path_dir, learning_rate, option):
        self.write_file_info(path_dir, ["learning_rate: "+str(learning_rate), "optimizer: " + str(option)])
        print("[+] summary network:")
        print(model.summary())
        
#         model =  tf.contrib.tpu.keras_to_tpu_model(model, \
#                                                    strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR']))
#                                                    )
        
        if option == "Adadelta":
            model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                        optimizer=tf.train.AdadeltaOptimizer(learning_rate=learning_rate, ),
                        metrics=['accuracy'])

        elif option == "SGD":
            model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                        optimizer=tf.keras.optimizers.SGD(lr=learning_rate),
                        metrics=['accuracy'])

        elif option == "Nadam":
            model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                        optimizer=tf.keras.optimizers.Nadam(lr=learning_rate),
                        metrics=['accuracy'])

        elif option == "RMSprop":
            model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                        optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate),
                        metrics=['accuracy'])


        return model

    def load_model_net(self, path):
        model = load_model(path)
        return model

    def last_load_model_net(self, experimental_path):
        weights = os.listdir(experimental_path)
        weights = sorted(weights, reverse=True)
        print("[+] WEIGHTS LIST: ", weights)
        print("[+] WEIGHTS RESTORE: ", weights[0])        
        model = load_model(os.path.join(experimental_path, weights[0]))
        return model
    
    
    def classify_image(self, model, img):
        return model.predict([img])

    def classify_image_from_path(self, model, path, dim):
        img = cv2.imread(path)
        img = cv2.resize(img, (dim, dim))
        img = np.array(img)
        img = img.astype('float32')
        img = img / 255

        img = img.reshape(1, dim, dim, 3)

        print("shape: ", img.shape)
        return model.predict([np.array(img)])

    def classify_bath_image_from_path(self, model, path, clazz, dim):
        cont = [0, 0]
        # print("255")
        for file in os.listdir(path):
            img = cv2.imread(os.path.join(path, file))
            img = cv2.resize(img, (dim, dim))
            cv2.imshow("img", img)
            cv2.waitKey(200)
            img = np.array(img)
            img = img.astype('float32')
            img = img / 255

            img = img.reshape(1, dim, dim, 3)
            # print([np.array(img)])
            # print("shape: ", img.shape)
            vet = model.predict([np.array(img)])
            # print("vet: ", vet)
            if np.argmax(vet[0]) == clazz:
                cont[0] += 1
            else:
                cont[1] += 1
                print(file)
        print("Classe: ", clazz, " Conts: ", cont)


# **`Neural Networks Architectures`**


In [0]:
import tensorflow.keras.applications as classifiers


class InceptionResNetV2:
    def build_network(self, input_shape, num_classes):
        model = classifiers.inception_resnet_v2.InceptionResNetV2(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class InceptionV3:
    def build_network(self, input_shape, num_classes):
        model = classifiers.inception_v3.InceptionV3(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class MobileNet:
    def build_network(self, input_shape, num_classes):
        model = classifiers.mobilenet.MobileNet(input_shape=input_shape,
                                                alpha=1.0,
                                                depth_multiplier=1,
                                                dropout=1e-3,
                                                include_top=True,
                                                weights=None,
                                                input_tensor=None,
                                                pooling='max',
                                                classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class MobileNetV2:
    def build_network(self, input_shape, num_classes):
        model = classifiers.mobilenetv2.MobileNetV2(input_shape=input_shape,
                                                alpha=1.0,
                                                depth_multiplier=1,
                                                include_top=True,
                                                weights=None,
                                                input_tensor=None,
                                                pooling='max',
                                                classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class ResNet50:
    def build_network(self, input_shape, num_classes):
        model = classifiers.resnet50.ResNet50(include_top=True,
                                        input_shape=input_shape,
                                        weights=None,
                                        input_tensor=None,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class VGG16:
    def build_network(self, input_shape, num_classes):
        model = classifiers.vgg16.VGG16(include_top=True,
                                        input_tensor=None,
                                        weights=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


In [0]:
import tensorflow.keras.applications as classifiers


class VGG19:
    def build_network(self, input_shape, num_classes):
        model = classifiers.vgg16.VGG16(include_top=True,
                                        weights=None,
                                        input_tensor=None,
                                        input_shape=input_shape,
                                        pooling='max',
                                        classes=num_classes)
        return model


# **`Utils Class`**

In [0]:
import os
import time
import datetime

class Utils:
    def make_dirs(self, path):
        if os.path.exists(path) == False:
            os.makedirs(path)

    def get_date_string(self):
        ts = time.time()
        st = datetime.datetime.fromtimestamp(ts).strftime('%H_%M_%S+%Y-%m-%d')
        return st


# **`Pre Processing Class`**

In [0]:
import cv2
import numpy as np
# from utils.constants import getDataPath
# from utils.constants import getDataAndLabelsPath
# from utils.constants import getLabelsPath
import os

class PreProcessing:

    def load_base_input_experiment(self, path):
        pwd = os.path.join(path, "training")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

#         x_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)
        x_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "training")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

#         y_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)
        y_train = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "testing")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

        x_test = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "testing")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

        y_test = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        return x_train, y_train, x_test, y_test


    def load_base(self, path):
        pwd = os.path.join(path, "images")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-data-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-data-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-data.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_data.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-data.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-data.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-data.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-data.npy'))

        images = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        pwd = os.path.join(path, "labels")
        CIFE_TR_DATA = np.load(os.path.join(pwd, 'CIFE-label-tr.npy'))
        CIFE_TS_DATA = np.load(os.path.join(pwd, 'CIFE-label-ts.npy'))
        CK_DATA = np.load(os.path.join(pwd, 'ck+-label.npy'))
        FER_DATA = np.load(os.path.join(pwd, 'fer_labels.npy'))
        JAFFE_DATA = np.load(os.path.join(pwd, 'JAFFE-label.npy'))
        KDEF_DATA = np.load(os.path.join(pwd, 'KDEF-label.npy'))
        NOVAEMOTIONS_DATA = np.load(os.path.join(pwd, 'novaemotions-label.npy'))
        RAFD_DATA = np.load(os.path.join(pwd, 'RafD-label.npy'))

        labels = np.concatenate((CIFE_TR_DATA, CIFE_TS_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA), axis=0)

        return images, labels


    def generate_images_with_size(self, size, save=True, paths=None, path_output=None, faceDetector=None):
        # CIFE_DATA, CK_DATA, FER_DATA, JAFFE_DATA, KDEF_DATA, NOVAEMOTIONS_DATA, RAFD_DATA = getDataPath()
        if paths == None:
            paths_images = getDataPath()
            path_labels = getLabelsPath()

        total_images = 0

        if len(paths_images) != len(path_labels):
            print("[+] ERROR PATH LABEL AND IMAGE SIZE DIFFERENCE")
            return False

        for index in range(len(paths_images)):
            print("path: ", paths_images[index])
            total_images += self.resize_all_images_label(size, paths_images[index], path_output, path_labels[index], faceDetector, save)

        print("[+] Total Images: ", total_images)
        return True

    # def generate_labels(self, save=False, paths=None, path_output=None):
    #     if paths == None:
    #         paths = getLabelsPath()
    #
    #     labels = []
    #
    #     # for path in enumerate(paths):
    #     for path in paths:
    #         label = np.load(path).tolist()
    #         labels += label
    #
    #     print("[+] Labels len: ", len(labels))
    #
    #     if save == True:
    #         self.make_dirs(os.path.join(path_output, "labels"))
    #         file_name = os.path.join(path_output, "labels",  "labels.npy")
    #         print("[+] File Saving: ", file_name)
    #         np.save(file_name, labels)
    #
    #
    #     return labels

    def load_from_file(self, path):
        return np.load(path)

    def resize_all_images_label(self, size, path_load, path_output, path_label, faceDetector=None, save=True):
        #TODO: Verify others possible resize processing, cubic, linear, etc, upsample
        new_images = []
        new_labels = []

        images = np.load(path_load)
        labels = np.load(path_label)

        print("[+] SIZE: ", size)

        if len(images) != len(labels):
            print("[+] ERROR LABEL AND IMAGE SIZE DIFFERENCE")
            return False

        for index in range(len(images)):
            coordinates = faceDetector.detectCNN(images[index])

            if coordinates != []:
                bbox = coordinates[0]
                # cv2.imshow("img1", images[index])
                # cv2.waitKey(25)
                print(index)
                img = images[index]
                img = img[bbox[0]:bbox[2], bbox[3]:bbox[1]]

                img = cv2.resize(img, (size, size))

                cv2.imshow("img-cropped-RSZ", img)
                cv2.waitKey(10)

                new_labels.append(labels[index])
                new_images.append(img)

        print("resized ", path_load, " :", len(new_images))
        print("labels ", path_label, " :", len(new_labels))

        if save == True:
            base_label = os.path.basename(os.path.normpath(path_load))
            # base_label = base_label.replace(".npy", "")

            self.make_dirs(os.path.join(path_output, "images"))
            file_image_name = os.path.join(path_output, "images", base_label)
            print("[+] File Saving: ", file_image_name)
            np.save(file_image_name, new_images)

            base_label = os.path.basename(os.path.normpath(path_label))
            # base_label = base_label.replace(".npy", "")

            self.make_dirs(os.path.join(path_output, "labels"))
            file_label_name = os.path.join(path_output, "labels",  base_label)
            print("[+] File Saving: ", file_label_name)
            np.save(file_label_name, new_labels)


        return len(new_images)

    def save_images_labels_size(self, size, faceDetector, path_output):
        # images_paths, labels_paths = getDataAndLabelsPath()
        # images = []
        # labels = []
        #
        # # for path in enumerate(paths):
        # for path in labels_paths:
        #     label = np.load(path).tolist()
        #     labels += label
        #     print("len_label: ", len(label))
        # print("len: ", len(labels))

        # self.generate_labels(save=True, path_output=path_output)
        self.generate_images_with_size(size, save=True, faceDetector=faceDetector, path_output=path_output)

    def make_dirs(self, path):
        if os.path.exists(path) == False:
            os.makedirs(path)


# Loading Database

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
import numpy as np
import os

preProcessing = PreProcessing()

IMG_SIZE = 185

ROOT_INPUT_DATA = "/content/drive/My Drive/base_de_dados"
BASE_DIR = "output_"+str(IMG_SIZE)


input_path = os.path.join(ROOT_INPUT_DATA, BASE_DIR)
print("[+] LOADING BASE INPUT")
x_train, y_train, x_test, y_test = preProcessing.load_base_input_experiment(input_path)
print("[+] PERFECT, IT'S LOADED THE INPUT BASE")


[+] LOADING BASE INPUT
[+] PERFECT, IT'S LOADED THE INPUT BASE


# **`Train Script`**


In [1]:
import tensorflow as tf

utils = Utils()

LEARNING_RATE = 0.01
BATCH_SIZE = 20


OPTIMIZERS = ["Adadelta", "SGD", "Nadam", "RMSprop"]
OPTIMIZER = OPTIMIZERS[0]

ARCHITECTURES = ["VGG16", "VGG19", "MobileNet", "MobileNetV2", \
                "ResNet50", "InceptionV3", "InceptionResNetV2"]

NUM_CLASSES = 7
EPOCHS = 1000

ARCHITECTURE = ARCHITECTURES[6]

ROOT_OUTPUT = "/content/drive/My Drive/experimentos_colab_SSFER"
EXPERIMENT_ID = ARCHITECTURE + "+" + str(IMG_SIZE) + "+" + utils.get_date_string()
output_path = os.path.join(ROOT_OUTPUT, EXPERIMENT_ID)
utils.make_dirs(output_path)
# output_path = drive.create_dir(top=ROOT_OUTPUT, name=EXPERIMENT_ID)


common = Common()
common.write_file_info(output_path, ["architecture: " + ARCHITECTURE, "img_size: " + str(IMG_SIZE)])

input_shape = (IMG_SIZE, IMG_SIZE, 3)

#NORMALIZE IMAGES DATASET
# x_train = x_train.astype('float32')
# x_train /= 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

print("[+] TYPE NDARRAY: ", x_train.dtype)

with tf.device('/gpu:0'):
    print("[+] " + ARCHITECTURE)
    if ARCHITECTURE == "VGG16":
        net = VGG16()
    elif ARCHITECTURE == "VGG19":
        net = VGG19()
    elif ARCHITECTURE == "MobileNet":
        net = MobileNet()
    elif ARCHITECTURE == "MobileNetV2":
        net = MobileNetV2()
    elif ARCHITECTURE == "ResNet50":
        net = ResNet50()
    elif ARCHITECTURE == "InceptionV3":
        net = InceptionV3()
    elif ARCHITECTURE == "InceptionResNetV2":
        net = InceptionResNetV2()


    print("[+] BUILD NET")
    model = net.build_network(input_shape, NUM_CLASSES)

    print("[+] COMPILE")
    model = common.compile_net(model, output_path, LEARNING_RATE, OPTIMIZER)

    print("[+] TRAIN")
    model, history = common.train(model, x_train, y_train, x_test, y_test, BATCH_SIZE, EPOCHS, output_path)

    print("[+] SAVE ON HISTORY")
    common.save_plot_losses(history, output_path)
    common.save_plot_acc(history, output_path)

NameError: ignored

# Restore model and Train

In [0]:
import tensorflow as tf

common = Common()

RECOVERY_PATH = "InceptionResNetV2+185+19_57_04+2018-12-04"

LEARNING_RATE = 0.01
BATCH_SIZE = 20


OPTIMIZERS = ["Adadelta", "SGD", "Nadam", "RMSprop"]
OPTIMIZER = OPTIMIZERS[0]

ARCHITECTURES = ["VGG16", "VGG19", "MobileNet", "MobileNetV2", \
                "ResNet50", "InceptionV3", "InceptionResNetV2"]

# NUM_CLASSES = 7
EPOCHS = 1000

ARCHITECTURE = ARCHITECTURES[6]

ROOT_OUTPUT = "/content/drive/My Drive/experimentos_colab_SSFER"

output_path = os.path.join(ROOT_OUTPUT, RECOVERY_PATH)

#NORMALIZE IMAGES DATASET
# x_train = x_train.astype('float32')
# x_train /= 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

print("[+] TYPE NDARRAY: ", x_train.dtype)

with tf.device('/gpu:0'):
#     print("[+] " + ARCHITECTURE)
#     if ARCHITECTURE == "VGG16":
#         net = VGG16()
#     elif ARCHITECTURE == "VGG19":
#         net = VGG19()
#     elif ARCHITECTURE == "MobileNet":
#         net = MobileNet()
#     elif ARCHITECTURE == "MobileNetV2":
#         net = MobileNetV2()
#     elif ARCHITECTURE == "ResNet50":
#         net = ResNet50()
#     elif ARCHITECTURE == "InceptionV3":
#         net = InceptionV3()
#     elif ARCHITECTURE == "InceptionResNetV2":
#         net = InceptionResNetV2()


    print("[+] RESTORE LAST MODEL")
    model = common.last_load_model_net(output_path)

    print("[+] COMPILE")
    model = common.compile_net(model, output_path, LEARNING_RATE, OPTIMIZER)

    print("[+] TRAIN")
    model, history = common.train(model, x_train, y_train, x_test, y_test, BATCH_SIZE, EPOCHS, output_path)

    print("[+] SAVE ON HISTORY")
    common.save_plot_losses(history, output_path)
    common.save_plot_acc(history, output_path)

x_train shape: (33519, 185, 185, 3)
33519 train samples
[+] TYPE NDARRAY:  uint8
[+] RESTORE LAST MODEL
[+] WEIGHTS LIST:  ['weights.08-1.30.hdf5', 'weights.07-1.39.hdf5', 'weights.06-1.39.hdf5', 'weights.05-1.46.hdf5', 'weights.04-1.49.hdf5', 'weights.03-1.59.hdf5', 'weights.02-1.69.hdf5', 'weights.01-1.76.hdf5', 'info.txt']
[+] WEIGHTS RESTORE:  weights.08-1.30.hdf5
[+] COMPILE
[+] summary network:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 185, 185, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 92, 92, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
ba